In [1]:
import pandas as pd
import numpy as np
import pingouin as pg
import numpy as np, statsmodels.stats.api as sms
import requests
from bs4 import BeautifulSoup 

with open('C:/Users/USER/Desktop/연구 결과물/청년가구/청년가구 gb AUC 부트스트랩 95% 신뢰 구간.html', 'r', encoding='utf-8') as f:
    html_content = f.read()

In [2]:
from selenium import webdriver

In [3]:
options = webdriver.ChromeOptions() 
options.add_argument('lang=ko_KR') 
options.add_argument('disable-gpu')
driver = webdriver.Chrome(options=options)

### XGBClassifier

In [4]:
driver.get(url='.html')

In [5]:
df = pd.DataFrame()

In [6]:
import warnings
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)

In [7]:
for i in range(0, 800):
    xpath = f'/html/body/div[{i}]/div[2]/div[2]/div/div[2]/pre'
    try:
        element = driver.find_element("xpath", xpath)
        text = element.text.strip('[]')
        df[f'Column_{i}'] = [text]
    except:
        df[f'Column_{i}'] = [None]

df = df.dropna(axis=1, how='all')

In [8]:
cols_to_drop = [col for col in df.columns if df[col].apply(
    lambda x: str(x).startswith('(') or str(x).startswith('{') or 
              str(x).startswith('array') or str(x).startswith('ShapiroResult') or
            str(x).startswith('BartlettResult')).any()]
df = df.drop(columns=cols_to_drop)

In [9]:
cols_to_drop_2 = [col for col in df.columns if not df[col].apply(lambda x: ',' in str(x)).any()]

df = df.drop(columns=cols_to_drop_2)

In [10]:
cols_to_drop_3 = [col for col in df.columns if df[col].apply(lambda x: 'alternative' in str(x)).any()]

df = df.drop(columns=cols_to_drop_3)

In [11]:
df

,Column_28,Column_42,Column_55,Column_67,Column_80,Column_92,Column_105,Column_118,Column_132,Column_145,...,Column_664,Column_677,Column_690,Column_703,Column_716,Column_729,Column_742,Column_755,Column_768,Column_781
0,"0.7667410714285714, 0.7585921479835953, 0.7654...","0.7609310919343815, 0.7544162252221464, 0.7649...","0.767966613978127, 0.7594892771701983, 0.76086...","0.76809744531784, 0.772067775974026, 0.7649708...","0.767392558099795, 0.7647492310321258, 0.76342...","0.7639455528024608, 0.7643433868762817, 0.7676...","0.755137132604238, 0.7698569933356119, 0.76675...","0.7672830869788108, 0.771600521189337, 0.75977...","0.7761529178058784, 0.7694404690704033, 0.7698...","0.7708796138072453, 0.757948671394395, 0.76040...",...,"0.7787294941900205, 0.7747818587662338, 0.7704...","0.7769392408578263, 0.7862696514012304, 0.7662...","0.768248301862611, 0.7762984342959671, 0.77291...","0.7678544728298018, 0.7695299149863295, 0.7717...","0.7635009932501708, 0.7701066408920028, 0.7680...","0.7516781121838688, 0.7547940340909091, 0.7625...","0.7452219860731374, 0.7494566494360904, 0.7424...","0.7513336786568695, 0.7510666759227613, 0.7438...","0.7307344177204375, 0.734441750683527, 0.73177...","0.706453189080656, 0.7057963623547504, 0.70127..."


In [12]:
all_data = []

for column in df.columns:
    column_data = list(map(float, df[column].iloc[0].split(', ')))
    all_data.append(column_data)

all_data_array = np.array(all_data).T
new_df = pd.DataFrame(all_data_array, columns=[f'xgb_{i}' for i in range(len(df.columns))])

In [13]:
len_all_data = []
len_all_data = [len(sublist) for sublist in all_data]
sum(len_all_data) # 59*2000

118000

In [14]:
pd.set_option('display.precision', 16)

In [15]:
new_df.shape

(2000, 59)

In [16]:
new_df = new_df.astype(float)

In [17]:
new_df

,xgb_0,xgb_1,xgb_2,xgb_3,xgb_4,xgb_5,xgb_6,xgb_7,xgb_8,xgb_9,...,xgb_49,xgb_50,xgb_51,xgb_52,xgb_53,xgb_54,xgb_55,xgb_56,xgb_57,xgb_58
0,0.7667410714285714,0.7609310919343815,0.7679666139781270,0.7680974453178400,0.7673925580997950,0.7639455528024608,0.7551371326042380,0.7672830869788108,0.7761529178058784,0.7708796138072453,...,0.7787294941900205,0.7769392408578263,0.7682483018626109,0.7678544728298018,0.7635009932501708,0.7516781121838688,0.7452219860731374,0.7513336786568695,0.7307344177204375,0.7064531890806560
1,0.7585921479835953,0.7544162252221464,0.7594892771701983,0.7720677759740260,0.7647492310321258,0.7643433868762817,0.7698569933356119,0.7716005211893370,0.7694404690704033,0.7579486713943950,...,0.7747818587662338,0.7862696514012304,0.7762984342959671,0.7695299149863295,0.7701066408920028,0.7547940340909091,0.7494566494360904,0.7510666759227613,0.7344417506835270,0.7057963623547504
2,0.7654621283321941,0.7649841934381407,0.7608643412508544,0.7649708433014354,0.7634275674982912,0.7676916011619959,0.7667544215652768,0.7597776401230349,0.7698409731715653,0.7604050965481887,...,0.7704524094326726,0.7662297611927547,0.7729115046138073,0.7717927631578947,0.7680734150717703,0.7625170881749829,0.7424211273923444,0.7438135466507176,0.7317717233424469,0.7012786760936431
3,0.7725590610047847,0.7730610261449078,0.7620044429254955,0.7770687371838687,0.7595880681818181,0.7636465097402596,0.7654861585782637,0.7599298316814764,0.7634836380724538,0.7708742737525631,...,0.7703669685577580,0.7758645548530416,0.7694911995898839,0.7675447496582366,0.7634182224025974,0.7561864533492823,0.7592743399692413,0.7494219390806561,0.7350558569719754,0.7064531890806560
4,0.7613289260082022,0.7618202110389610,0.7610832834928229,0.7639241925837321,0.7562024735133288,0.7729141746411483,0.7604024265208476,0.7678704929938482,0.7605973385167464,0.7687008714969241,...,0.7692228618421052,0.7655489042207793,0.7681134654818865,0.7752424384825701,0.7729729152426521,0.7602849453178400,0.7480976055194806,0.7498972039473685,0.7267307117224879,0.7059245236671222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.7570168318523581,0.7621432843472318,0.7647305408407381,0.7780619873547505,0.7628641917293233,0.7732853084415585,0.7648293318523580,0.7663565874914560,0.7671736158578264,0.7685833902939166,...,0.7657638414217363,0.7684044984620643,0.7664713986671224,0.7696206959159263,0.7717353575700615,0.7521827473513328,0.7506861970266575,0.7441326149179767,0.7298613187799042,0.7036416502904989
1996,0.7680974453178401,0.7713148282638415,0.7680787551264525,0.7662497863978127,0.7777095437457280,0.7550196514012304,0.7628001110731375,0.7664126580656186,0.7646664601845523,0.7699744745386192,...,0.7784758415926180,0.7667904669343815,0.7627333603896104,0.7695753054511280,0.7750675516917294,0.7557766041524265,0.7495487653793574,0.7534109599282296,0.7329772406869447,0.7014989533492821
1997,0.7649628332194122,0.7581542634996583,0.7669253033151059,0.7780646573820915,0.7733894395078605,0.7631632347915243,0.7710291353383458,0.7759419856459331,0.7616546693438141,0.7798081852358170,...,0.7711359364319892,0.7700839456596036,0.7588618207450445,0.7724602699931647,0.7712707728127135,0.7581822987867396,0.7471217105263157,0.7442007006151743,0.7326528323650034,0.7057963623547504
1998,0.7620418233082706,0.7665314742822966,0.7675874700956938,0.7628001110731373,0.7753866199589884,0.7669573436431989,0.7615371881408066,0.7647091806220094,0.7720170454545454,0.7649414730006835,...,0.7702348022043745,0.7740836466165413,0.7685620300751881,0.7690960355434040,0.7702868677375256,0.7528248889268627,0.7465289644565960,0.7504579096889953,0.7318945446001365,0.7012786760936431


In [18]:
driver.close()
driver.quit()

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=124.0.6367.119)
Stacktrace:
	GetHandleVerifier [0x00007FF7F3C61572+60802]
	(No symbol) [0x00007FF7F3BDAC72]
	(No symbol) [0x00007FF7F3A97CE4]
	(No symbol) [0x00007FF7F3A7E894]
	(No symbol) [0x00007FF7F3A7F3FF]
	(No symbol) [0x00007FF7F3A982DD]
	(No symbol) [0x00007FF7F3A6DACC]
	(No symbol) [0x00007FF7F3B19E47]
	(No symbol) [0x00007FF7F3B1984B]
	(No symbol) [0x00007FF7F3B0AB40]
	(No symbol) [0x00007FF7F3AD8FEC]
	(No symbol) [0x00007FF7F3AD9C21]
	GetHandleVerifier [0x00007FF7F3F6420D+3217949]
	GetHandleVerifier [0x00007FF7F3FA61A7+3488183]
	GetHandleVerifier [0x00007FF7F3F9F12F+3459391]
	GetHandleVerifier [0x00007FF7F3D1B936+823622]
	(No symbol) [0x00007FF7F3BE600F]
	(No symbol) [0x00007FF7F3BE0F34]
	(No symbol) [0x00007FF7F3BE10C2]
	(No symbol) [0x00007FF7F3BD1914]
	BaseThreadInitThunk [0x00007FF9458B7344+20]
	RtlU

In [19]:
print('AUC 신뢰 구간')
for column in new_df.columns:
    percentiles = np.percentile(new_df[column], [2.5, 97.5])
    rounded_percentiles = np.round(percentiles, 8)
    print(f"{column}: [{rounded_percentiles[0]} {rounded_percentiles[1]}]")
    print()

AUC 신뢰 구간
xgb_0: [0.75485932 0.77674339]

xgb_1: [0.75445267 0.77591328]

xgb_2: [0.75524854 0.77664417]

xgb_3: [0.75468803 0.77629823]

xgb_4: [0.75459218 0.77629189]

xgb_5: [0.75465546 0.77739188]

xgb_6: [0.75493575 0.77663092]

xgb_7: [0.75489943 0.77710809]

xgb_8: [0.75536662 0.77721058]

xgb_9: [0.75543244 0.77636699]

xgb_10: [0.75485751 0.77623839]

xgb_11: [0.75463957 0.77652672]

xgb_12: [0.75479744 0.77660455]

xgb_13: [0.75721174 0.77883957]

xgb_14: [0.76028001 0.77976059]

xgb_15: [0.76048466 0.77900257]

xgb_16: [0.75945216 0.77819969]

xgb_17: [0.75465966 0.77610259]

xgb_18: [0.7597751 0.77758145]

xgb_19: [0.76046484 0.77829164]

xgb_20: [0.7554044 0.77603597]

xgb_21: [0.75498941 0.77631445]

xgb_22: [0.75471013 0.77707715]

xgb_23: [0.75552142 0.7767484]

xgb_24: [0.75623338 0.77697909]

xgb_25: [0.75552475 0.77727596]

xgb_26: [0.76294409 0.7809617]

xgb_27: [0.76161996 0.78088708]

xgb_28: [0.76064727 0.77893802]

xgb_29: [0.75608132 0.77756089]

xgb_30: [0.756

In [20]:
print('AUC 신뢰 구간')
for column in new_df.columns:
    percentiles = np.percentile(new_df[column], [2.5, 97.5])
    rounded_percentiles = np.round(percentiles, 2)
    print(f"{column}: [{rounded_percentiles[0]} {rounded_percentiles[1]}]")

AUC 신뢰 구간
xgb_0: [0.75 0.78]
xgb_1: [0.75 0.78]
xgb_2: [0.76 0.78]
xgb_3: [0.75 0.78]
xgb_4: [0.75 0.78]
xgb_5: [0.75 0.78]
xgb_6: [0.75 0.78]
xgb_7: [0.75 0.78]
xgb_8: [0.76 0.78]
xgb_9: [0.76 0.78]
xgb_10: [0.75 0.78]
xgb_11: [0.75 0.78]
xgb_12: [0.75 0.78]
xgb_13: [0.76 0.78]
xgb_14: [0.76 0.78]
xgb_15: [0.76 0.78]
xgb_16: [0.76 0.78]
xgb_17: [0.75 0.78]
xgb_18: [0.76 0.78]
xgb_19: [0.76 0.78]
xgb_20: [0.76 0.78]
xgb_21: [0.75 0.78]
xgb_22: [0.75 0.78]
xgb_23: [0.76 0.78]
xgb_24: [0.76 0.78]
xgb_25: [0.76 0.78]
xgb_26: [0.76 0.78]
xgb_27: [0.76 0.78]
xgb_28: [0.76 0.78]
xgb_29: [0.76 0.78]
xgb_30: [0.76 0.78]
xgb_31: [0.76 0.78]
xgb_32: [0.76 0.78]
xgb_33: [0.76 0.78]
xgb_34: [0.76 0.78]
xgb_35: [0.76 0.78]
xgb_36: [0.76 0.78]
xgb_37: [0.76 0.78]
xgb_38: [0.77 0.78]
xgb_39: [0.76 0.78]
xgb_40: [0.76 0.78]
xgb_41: [0.76 0.78]
xgb_42: [0.76 0.78]
xgb_43: [0.76 0.78]
xgb_44: [0.76 0.78]
xgb_45: [0.76 0.78]
xgb_46: [0.76 0.78]
xgb_47: [0.76 0.78]
xgb_48: [0.76 0.78]
xgb_49: [0.76 0.78]


In [21]:
import pingouin as pg
import numpy as np, statsmodels.stats.api as sms

In [22]:
i = 0
while i < 58:  
    base_column = f'xgb_{i}'
    continue_search = True  

    for j in range(i + 1, 59):
        if not continue_search:
            break  
        compare_column = f'xgb_{j}'
        base_stats = sms.DescrStatsW(new_df[base_column])
        compare_stats = sms.DescrStatsW(new_df[compare_column])
        cm = sms.CompareMeans(base_stats, compare_stats)
        confint = cm.tconfint_diff(usevar='unequal')

        print(f'{base_column} and {compare_column}:')
        print(confint)
        print(cm.summary())
        print('-------------------------------------------')
        
        if confint[0] <= 0 <= confint[1] or (confint[0] < 0 and confint[1] < 0):
            continue_search = False  
        if compare_column == 'xgb_58':
            break
    if compare_column == 'xgb_58' and continue_search:
        break
    i += 1  

xgb_0 and xgb_1:
(-0.0001630042593546774, 0.0005275350821502053)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1      0.0002      0.000      1.035      0.301      -0.000       0.001
-------------------------------------------
xgb_1 and xgb_2:
(-0.0006220758095644087, 6.64444548993327e-05)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1     -0.0003      0.000     -1.582      0.114      -0.001    6.64e-05
-------------------------------------------
xgb_2 and xgb_3:
(2.344342772512474e-05, 0.0007117259054089866)
                          Test for equality of means                          
        

In [23]:
## xgboost는 X = 38

In [24]:
del new_df

### LGBMClassifier

In [39]:
driver = webdriver.Chrome(options=options)
driver.get(url='.html')

In [40]:
df = pd.DataFrame()

In [41]:
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)

In [42]:
for i in range(0, 800):
    xpath = f'/html/body/div[{i}]/div[2]/div[2]/div/div[2]/pre'
    try:
        element = driver.find_element("xpath", xpath)
        text = element.text.strip('[]')
        df[f'Column_{i}'] = [text]
    except:
        df[f'Column_{i}'] = [None]

df = df.dropna(axis=1, how='all')

In [43]:
cols_to_drop = [col for col in df.columns if df[col].apply(
    lambda x: str(x).startswith('(') or str(x).startswith('{') or 
              str(x).startswith('array') or str(x).startswith('ShapiroResult') or
            str(x).startswith('BartlettResult')
).any()]

df = df.drop(columns=cols_to_drop)

In [44]:
cols_to_drop_2 = [col for col in df.columns if not df[col].apply(lambda x: ',' in str(x)).any()]

df = df.drop(columns=cols_to_drop_2)

In [45]:
cols_to_drop_3 = [col for col in df.columns if df[col].apply(lambda x: 'alternative' in str(x)).any()]

df = df.drop(columns=cols_to_drop_3)

In [46]:
df

,Column_30,Column_45,Column_59,Column_72,Column_85,Column_97,Column_110,Column_123,Column_136,Column_149,...,Column_671,Column_684,Column_697,Column_710,Column_723,Column_736,Column_749,Column_762,Column_775,Column_788
0,"0.7684872693096376, 0.7691440960355433, 0.7687...","0.7593851461038962, 0.7702601674641147, 0.7674...","0.7666663106630212, 0.7661296351674642, 0.7647...","0.7709997650375939, 0.76953125, 0.769699461722...","0.7774745813397128, 0.7651844454887218, 0.7675...","0.76715759569378, 0.7655982997265893, 0.763128...","0.761454417293233, 0.7704844497607656, 0.76243...","0.7681989063568011, 0.7650322539302803, 0.7611...","0.7764145804853042, 0.7725510509227614, 0.7777...","0.7649307928913193, 0.7672190063226249, 0.7582...",...,"0.7813594711209844, 0.7742011278195489, 0.7719...","0.7748446044087491, 0.7798108552631579, 0.7697...","0.7659280481032125, 0.7762330186261107, 0.7738...","0.7517582130041012, 0.7592863550922762, 0.7527...","0.7643780972317157, 0.7696166908749146, 0.7663...","0.7505580357142858, 0.7508624188311688, 0.7572...","0.7461204502734109, 0.752400354579631, 0.73996...","0.7493044578776487, 0.7473406527682843, 0.7359...","0.7237683163875598, 0.729296607997266, 0.72455...","0.706453189080656, 0.706453189080656, 0.705768..."


In [47]:
all_data = []

for column in df.columns:
    column_data = list(map(float, df[column].iloc[0].split(', ')))
    all_data.append(column_data)

all_data_array = np.array(all_data).T
new_df = pd.DataFrame(all_data_array, columns=[f'lgbm_{i}' for i in range(len(df.columns))])

In [48]:
len_all_data = []
len_all_data = [len(sublist) for sublist in all_data]
sum(len_all_data) # 2000 * 59

118000

In [49]:
new_df.shape

(2000, 59)

In [50]:
new_df = new_df.astype(float)

In [51]:
new_df

,lgbm_0,lgbm_1,lgbm_2,lgbm_3,lgbm_4,lgbm_5,lgbm_6,lgbm_7,lgbm_8,lgbm_9,...,lgbm_49,lgbm_50,lgbm_51,lgbm_52,lgbm_53,lgbm_54,lgbm_55,lgbm_56,lgbm_57,lgbm_58
0,0.7684872693096376,0.7593851461038962,0.7666663106630212,0.7709997650375939,0.7774745813397128,0.7671575956937799,0.7614544172932330,0.7681989063568011,0.7764145804853042,0.7649307928913193,...,0.7813594711209844,0.7748446044087491,0.7659280481032125,0.7517582130041012,0.7643780972317157,0.7505580357142858,0.7461204502734109,0.7493044578776487,0.7237683163875598,0.7064531890806560
1,0.7691440960355433,0.7702601674641147,0.7661296351674642,0.7695312500000000,0.7651844454887218,0.7655982997265893,0.7704844497607656,0.7650322539302803,0.7725510509227614,0.7672190063226249,...,0.7742011278195489,0.7798108552631579,0.7762330186261107,0.7592863550922762,0.7696166908749146,0.7508624188311688,0.7524003545796309,0.7473406527682843,0.7292966079972660,0.7064531890806560
2,0.7687115516062885,0.7674833390293917,0.7647892814422419,0.7696994617224880,0.7675340695488722,0.7631285244360901,0.7624369873547505,0.7611206638755981,0.7777576042378674,0.7582397043745728,...,0.7719850051264524,0.7697688824333562,0.7738219839371155,0.7527034026828434,0.7663993079289133,0.7572918446684894,0.7399620322112099,0.7359876965140124,0.7245573094668489,0.7057683270676691
3,0.7690025845864663,0.7613022257347914,0.7721051563568011,0.7795945830485305,0.7609871625085441,0.7619296821599453,0.7623195061517430,0.7580474624060151,0.7652912465823650,0.7693630382775118,...,0.7747645035885169,0.7673765379357484,0.7699637944292549,0.7599845672419686,0.7604037615345181,0.7537914388243336,0.7566069826555024,0.7478279327580314,0.7279762794771018,0.7064531890806560
4,0.7638948222829802,0.7610352230006836,0.7648159817156528,0.7687062115516061,0.7640657040328093,0.7721318566302118,0.7569153708133971,0.7761742780246070,0.7698169429254955,0.7648239917976760,...,0.7732492630724537,0.7688076725905674,0.7652191558441559,0.7649521531100478,0.7760754870129869,0.7616466592617908,0.7472378567156528,0.7449523133116883,0.7163790157211209,0.7059245236671222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.7673551777170198,0.7655662593984963,0.7676488807245385,0.7783183099794940,0.7560689721462748,0.7724869702665755,0.7673551777170198,0.7583304853041695,0.7645436389268626,0.7716298914900888,...,0.7697995877477786,0.7671362354750513,0.7657958817498292,0.7545163512474368,0.7598043403964456,0.7544095501537935,0.7426000192241968,0.7402250299043063,0.7231288448393712,0.7064531890806560
1996,0.7696407211209844,0.7709383544087491,0.7705645505809979,0.7694030886876282,0.7703909988038277,0.7639028323650034,0.7666502904989747,0.7687249017429939,0.7692055066643882,0.7607762303485988,...,0.7816104536910459,0.7690119296821600,0.7590433826042379,0.7538982399179768,0.7646317498291182,0.7583371603725223,0.7530865516062883,0.7520479109706083,0.7300909411312371,0.7017793062200957
1997,0.7688023325358851,0.7642419258373205,0.7661910457963089,0.7786680835611757,0.7760060663021189,0.7562532040328093,0.7663245471633631,0.7753358894395079,0.7694004186602871,0.7693817284688995,...,0.7787375042720437,0.7656543703007519,0.7614917976760082,0.7597202345352017,0.7720878011790842,0.7648587021531100,0.7494179340396445,0.7369382262474367,0.7242382411995900,0.7065573201469583
1998,0.7682870172590568,0.7736324119958988,0.7665274692412849,0.7651710953520163,0.7745722616199590,0.7666235902255639,0.7684125085440875,0.7610565832194122,0.7720090353725222,0.7636545198222831,...,0.7801699739405332,0.7726124615516063,0.7637319506151743,0.7497556924982911,0.7767630190533150,0.7583465054682159,0.7484153387730691,0.7505099752221462,0.7224706830997950,0.7064531890806560


In [52]:
driver.close()
driver.quit()

In [53]:
print('AUC 신뢰 구간')
for column in new_df.columns:
    percentiles = np.percentile(new_df[column], [2.5, 97.5])
    rounded_percentiles = np.round(percentiles, 8)
    print(f"{column}: [{rounded_percentiles[0]} {rounded_percentiles[1]}]")

AUC 신뢰 구간
lgbm_0: [0.759642 0.77914455]
lgbm_1: [0.7599024 0.77882915]
lgbm_2: [0.75974246 0.77837545]
lgbm_3: [0.75935865 0.77841243]
lgbm_4: [0.75936926 0.7784681]
lgbm_5: [0.75534366 0.77648674]
lgbm_6: [0.75563015 0.77589172]
lgbm_7: [0.75513166 0.77564721]
lgbm_8: [0.76020164 0.77794471]
lgbm_9: [0.75148901 0.77473073]
lgbm_10: [0.76191046 0.77934794]
lgbm_11: [0.75650078 0.77701767]
lgbm_12: [0.7555078 0.77564835]
lgbm_13: [0.75485585 0.77541919]
lgbm_14: [0.7548965 0.77622808]
lgbm_15: [0.75657608 0.77773731]
lgbm_16: [0.75866864 0.77852924]
lgbm_17: [0.75631628 0.77634516]
lgbm_18: [0.75516337 0.77678698]
lgbm_19: [0.75215218 0.77495434]
lgbm_20: [0.75395391 0.77704804]
lgbm_21: [0.75543471 0.77712641]
lgbm_22: [0.75978411 0.77883122]
lgbm_23: [0.75767439 0.77682216]
lgbm_24: [0.76149453 0.780241]
lgbm_25: [0.75724512 0.77646311]
lgbm_26: [0.75745959 0.77843379]
lgbm_27: [0.75741981 0.77806326]
lgbm_28: [0.76066789 0.78006791]
lgbm_29: [0.75759549 0.77688224]
lgbm_30: [0.759985

In [54]:
print('AUC 신뢰 구간')
for column in new_df.columns:
    percentiles = np.percentile(new_df[column], [2.5, 97.5])
    rounded_percentiles = np.round(percentiles, 2)
    print(f"{column}: [{rounded_percentiles[0]} {rounded_percentiles[1]}]")

AUC 신뢰 구간
lgbm_0: [0.76 0.78]
lgbm_1: [0.76 0.78]
lgbm_2: [0.76 0.78]
lgbm_3: [0.76 0.78]
lgbm_4: [0.76 0.78]
lgbm_5: [0.76 0.78]
lgbm_6: [0.76 0.78]
lgbm_7: [0.76 0.78]
lgbm_8: [0.76 0.78]
lgbm_9: [0.75 0.77]
lgbm_10: [0.76 0.78]
lgbm_11: [0.76 0.78]
lgbm_12: [0.76 0.78]
lgbm_13: [0.75 0.78]
lgbm_14: [0.75 0.78]
lgbm_15: [0.76 0.78]
lgbm_16: [0.76 0.78]
lgbm_17: [0.76 0.78]
lgbm_18: [0.76 0.78]
lgbm_19: [0.75 0.77]
lgbm_20: [0.75 0.78]
lgbm_21: [0.76 0.78]
lgbm_22: [0.76 0.78]
lgbm_23: [0.76 0.78]
lgbm_24: [0.76 0.78]
lgbm_25: [0.76 0.78]
lgbm_26: [0.76 0.78]
lgbm_27: [0.76 0.78]
lgbm_28: [0.76 0.78]
lgbm_29: [0.76 0.78]
lgbm_30: [0.76 0.78]
lgbm_31: [0.76 0.78]
lgbm_32: [0.76 0.78]
lgbm_33: [0.76 0.78]
lgbm_34: [0.76 0.78]
lgbm_35: [0.76 0.78]
lgbm_36: [0.76 0.78]
lgbm_37: [0.76 0.78]
lgbm_38: [0.76 0.78]
lgbm_39: [0.75 0.78]
lgbm_40: [0.76 0.78]
lgbm_41: [0.75 0.78]
lgbm_42: [0.76 0.78]
lgbm_43: [0.76 0.78]
lgbm_44: [0.76 0.78]
lgbm_45: [0.75 0.77]
lgbm_46: [0.76 0.78]
lgbm_47: [0.7

In [55]:
i = 0
while i < 58:  
    base_column = f'lgbm_{i}'
    continue_search = True  

    for j in range(i + 1, 59):
        if not continue_search:
            break  
        compare_column = f'lgbm_{j}'
        base_stats = sms.DescrStatsW(new_df[base_column])
        compare_stats = sms.DescrStatsW(new_df[compare_column])
        cm = sms.CompareMeans(base_stats, compare_stats)
        confint = cm.tconfint_diff(usevar='unequal')

        print(f'{base_column} and {compare_column}:')
        print(confint)
        print(cm.summary())
        print('-------------------------------------------')
        
        if confint[0] <= 0 <= confint[1] or (confint[0] < 0 and confint[1] < 0):
            continue_search = False  
        if compare_column == 'lgbm_58':
            break
    if compare_column == 'lgbm_58' and continue_search:
        break
    i += 1  

lgbm_0 and lgbm_1:
(-0.0002441103990866831, 0.0003669223116806839)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1   6.141e-05      0.000      0.394      0.694      -0.000       0.000
-------------------------------------------
lgbm_1 and lgbm_2:
(-3.8867927446208095e-05, 0.0005624616240456973)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1      0.0003      0.000      1.707      0.088   -3.89e-05       0.001
-------------------------------------------
lgbm_2 and lgbm_3:
(-0.00018558969232100792, 0.0004097117873311121)
                          Test for equality of means                         

lgbm_49 and lgbm_52:
(0.015981930276389264, 0.01660528082665215)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1      0.0163      0.000    102.495      0.000       0.016       0.017
-------------------------------------------
lgbm_49 and lgbm_53:
(0.006725500525829717, 0.007270111284234873)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1      0.0070      0.000     50.383      0.000       0.007       0.007
-------------------------------------------
lgbm_49 and lgbm_54:
(0.01646919170303347, 0.01702707159370247)
                          Test for equality of means                          
      

In [56]:
### lgbm = 49

In [57]:
del new_df

### GradientBoosting

In [58]:
driver = webdriver.Chrome(options=options)
driver.get(url='.html')

In [59]:
df = pd.DataFrame()

In [60]:
for i in range(0, 775):
    xpath = f'/html/body/div[{i}]/div[2]/div[2]/div/div[2]/pre'
    try:
        element = driver.find_element("xpath", xpath)
        text = element.text.strip('[]')
        df[f'Column_{i}'] = [text]
    except:
        df[f'Column_{i}'] = [None]

df = df.dropna(axis=1, how='all')

In [61]:
cols_to_drop = [col for col in df.columns if df[col].apply(
    lambda x: str(x).startswith('(') or str(x).startswith('{') or 
              str(x).startswith('array') or str(x).startswith('ShapiroResult') or
            str(x).startswith('BartlettResult')
).any()]

df = df.drop(columns=cols_to_drop)

In [62]:
cols_to_drop_2 = [col for col in df.columns if not df[col].apply(lambda x: ',' in str(x)).any()]

df = df.drop(columns=cols_to_drop_2)

In [63]:
cols_to_drop_3 = [col for col in df.columns if df[col].apply(lambda x: 'alternative' in str(x)).any()]

df = df.drop(columns=cols_to_drop_3)

In [64]:
df

,Column_24,Column_36,Column_48,Column_60,Column_72,Column_83,Column_95,Column_107,Column_119,Column_131,...,Column_617,Column_630,Column_643,Column_656,Column_669,Column_682,Column_695,Column_708,Column_721,Column_734
0,"0.7639268626110732, 0.7570862525632263, 0.7553...","0.7605839883800409, 0.7631231843814081, 0.7659...","0.7639589029391661, 0.7542319933356118, 0.7651...","0.7594492267600821, 0.7659988038277512, 0.7705...","0.77353629101162, 0.7573051948051949, 0.759985...","0.7618068609022556, 0.7640123034859877, 0.7608...","0.7625704887218046, 0.7637186004784688, 0.7622...","0.7620338132262474, 0.7701747265892003, 0.7583...","0.7713174982911823, 0.7658679724880384, 0.7695...","0.7678571428571428, 0.7601060534859876, 0.7639...",...,"0.7748352593130553, 0.7684071684894054, 0.7678...","0.7735977016404647, 0.78592254784689, 0.768716...","0.7687769672761449, 0.7797587897300067, 0.7730...","0.7719035692925496, 0.7751556625939849, 0.7756...","0.7631458796138073, 0.7742985838174982, 0.7672...","0.7514751901059467, 0.7471444057587149, 0.7573...","0.7481470010252904, 0.7540664516404647, 0.7494...","0.7444249829118252, 0.7451031698564593, 0.7312...","0.7201103789302802, 0.7289067840054682, 0.7130...","0.7057362867395761, 0.7057362867395761, 0.7064..."


In [65]:
all_data = []

for column in df.columns:
    column_data = list(map(float, df[column].iloc[0].split(', ')))
    all_data.append(column_data)

all_data_array = np.array(all_data).T
new_df = pd.DataFrame(all_data_array, columns=[f'gb_{i}' for i in range(len(df.columns))])

In [66]:
new_df.shape

(2000, 59)

In [67]:
new_df = new_df.astype(float)

In [68]:
new_df

,gb_0,gb_1,gb_2,gb_3,gb_4,gb_5,gb_6,gb_7,gb_8,gb_9,...,gb_49,gb_50,gb_51,gb_52,gb_53,gb_54,gb_55,gb_56,gb_57,gb_58
0,0.7639268626110732,0.7605839883800409,0.7639589029391661,0.7594492267600821,0.7735362910116200,0.7618068609022556,0.7625704887218046,0.7620338132262474,0.7713174982911823,0.7678571428571428,...,0.7748352593130553,0.7735977016404647,0.7687769672761449,0.7719035692925496,0.7631458796138073,0.7514751901059467,0.7481470010252904,0.7444249829118252,0.7201103789302802,0.7057362867395761
1,0.7570862525632263,0.7631231843814081,0.7542319933356118,0.7659988038277512,0.7573051948051949,0.7640123034859877,0.7637186004784688,0.7701747265892003,0.7658679724880384,0.7601060534859876,...,0.7684071684894054,0.7859225478468900,0.7797587897300067,0.7751556625939849,0.7742985838174982,0.7471444057587149,0.7540664516404647,0.7451031698564593,0.7289067840054682,0.7057362867395761
2,0.7553507347915243,0.7659774436090225,0.7651123547505128,0.7705565404989747,0.7599859022556391,0.7608323009227614,0.7622020249487356,0.7583598556049214,0.7695739704374572,0.7639455528024607,...,0.7678024072966507,0.7687168916609706,0.7730783813226247,0.7756976781442242,0.7672136662679426,0.7573439102016404,0.7494032488892686,0.7312417229152426,0.7130428165584416,0.7064812243677374
3,0.7738139738550923,0.7606640892002734,0.7708662636705399,0.7700118549213945,0.7608990516062885,0.7636064593301435,0.7695739704374573,0.7556738080997949,0.7599405117908407,0.7650936645591250,...,0.7714803699589883,0.7679198884996582,0.7758084842788791,0.7637853511619959,0.7681281506322625,0.7526006066302119,0.7552893241626795,0.7439684082365003,0.7173388905502392,0.7064251537935747
4,0.7631044941900207,0.7647652511961723,0.7605839883800409,0.7587550196514012,0.7598016703691045,0.7684765892002734,0.7596681690020505,0.7726818822624745,0.7638761320915927,0.7630751238892686,...,0.7716432416267942,0.7713428635509227,0.7682469668489404,0.7779605263157895,0.7694110987696515,0.7652084757347914,0.7514244595864661,0.7456438503930281,0.7136876281613124,0.7064251537935747
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.7481576811346549,0.7691254058441559,0.7616092788790156,0.7769138755980861,0.7592676649008885,0.7629256023581681,0.7653232869104579,0.7696220309295967,0.7679639439507860,0.7621059039644565,...,0.7682923573137388,0.7688891084244702,0.7656530352870813,0.7676889311346549,0.7665621795967190,0.7501121411483254,0.7444209778708134,0.7349317007006152,0.7156634483937115,0.7030689294258372
1996,0.7641698350991115,0.7774906015037594,0.7632379955570745,0.7578979408749146,0.7669920539986330,0.7497944078947369,0.7678624829118249,0.7582103340738209,0.7694725093984962,0.7602128545796308,...,0.7782609043916610,0.7672750768967874,0.7733533941387559,0.7681735410970609,0.7680213495386193,0.7629362824675323,0.7514338046821600,0.7478546330314423,0.7252101311517429,0.7064251537935747
1997,0.7601888243335612,0.7633661568694463,0.7637426307245386,0.7656276700273410,0.7760995172590568,0.7566804084073819,0.7697395121326042,0.7769726161995898,0.7733840994531784,0.7811111585782637,...,0.7757617588004101,0.7677450017088174,0.7696954566814764,0.7724923103212576,0.7736484321599453,0.7646130596377306,0.7530545112781954,0.7323164089200275,0.7184336017600821,0.7050661098769651
1998,0.7554335056390977,0.7617774906015038,0.7659240430622010,0.7623702366712235,0.7774478810663021,0.7633795070061518,0.7716966421736159,0.7636999102870814,0.7775680322966507,0.7752771488380041,...,0.7748499444634311,0.7700986308099795,0.7704630895420369,0.7698449782125769,0.7666996860047848,0.7654728084415583,0.7501134761619959,0.7481016105604922,0.7190036525974025,0.7064251537935747


In [69]:
len_all_data = []
len_all_data = [len(sublist) for sublist in all_data]
sum(len_all_data) # 2000 * 59

118000

In [70]:
driver.close()
driver.quit()

In [71]:
print('AUC 신뢰 구간')
for column in new_df.columns:
    percentiles = np.percentile(new_df[column], [2.5, 97.5])
    rounded_percentiles = np.round(percentiles, 2)
    print(f"{column}: [{rounded_percentiles[0]} {rounded_percentiles[1]}]")

AUC 신뢰 구간
gb_0: [0.75 0.77]
gb_1: [0.75 0.78]
gb_2: [0.75 0.78]
gb_3: [0.75 0.78]
gb_4: [0.75 0.78]
gb_5: [0.75 0.77]
gb_6: [0.76 0.78]
gb_7: [0.75 0.78]
gb_8: [0.76 0.78]
gb_9: [0.75 0.78]
gb_10: [0.76 0.78]
gb_11: [0.75 0.77]
gb_12: [0.75 0.78]
gb_13: [0.75 0.78]
gb_14: [0.75 0.78]
gb_15: [0.75 0.78]
gb_16: [0.75 0.78]
gb_17: [0.75 0.78]
gb_18: [0.76 0.78]
gb_19: [0.75 0.77]
gb_20: [0.75 0.77]
gb_21: [0.75 0.78]
gb_22: [0.76 0.78]
gb_23: [0.76 0.78]
gb_24: [0.75 0.78]
gb_25: [0.76 0.78]
gb_26: [0.76 0.78]
gb_27: [0.76 0.78]
gb_28: [0.75 0.78]
gb_29: [0.76 0.78]
gb_30: [0.76 0.78]
gb_31: [0.75 0.78]
gb_32: [0.76 0.78]
gb_33: [0.76 0.78]
gb_34: [0.76 0.78]
gb_35: [0.76 0.78]
gb_36: [0.76 0.78]
gb_37: [0.76 0.78]
gb_38: [0.76 0.78]
gb_39: [0.76 0.78]
gb_40: [0.76 0.78]
gb_41: [0.76 0.78]
gb_42: [0.75 0.78]
gb_43: [0.76 0.78]
gb_44: [0.75 0.77]
gb_45: [0.76 0.78]
gb_46: [0.76 0.78]
gb_47: [0.75 0.78]
gb_48: [0.76 0.78]
gb_49: [0.76 0.78]
gb_50: [0.76 0.78]
gb_51: [0.76 0.78]
gb_52: [0.76

In [75]:
i = 0
while i < 58:  
    base_column = f'gb_{i}'
    continue_search = True  

    for j in range(i + 1, 59):
        if not continue_search:
            break  
        compare_column = f'gb_{j}'
        base_stats = sms.DescrStatsW(new_df[base_column])
        compare_stats = sms.DescrStatsW(new_df[compare_column])
        cm = sms.CompareMeans(base_stats, compare_stats)
        confint = cm.tconfint_diff(usevar='unequal')

        print(f'{base_column} and {compare_column}:')
        print(confint)
        print(cm.summary())
        print('-------------------------------------------')
        
        if confint[0] <= 0 <= confint[1] or (confint[0] < 0 and confint[1] < 0):
            continue_search = False  
        if compare_column == 'gb_58':
            break
    if compare_column == 'gb_58' and continue_search:
        break
    i += 1  

gb_0 and gb_1:
(-0.0048412681646091745, -0.004096413397249971)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1     -0.0045      0.000    -23.525      0.000      -0.005      -0.004
-------------------------------------------
gb_1 and gb_2:
(0.002102653594843979, 0.002818310925832495)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1      0.0025      0.000     13.481      0.000       0.002       0.003
-------------------------------------------
gb_1 and gb_3:
(0.0023374632252297894, 0.003066455840901235)
                          Test for equality of means                          
                 

In [76]:
## gb = 51

In [77]:
del new_df

### ExtraTrees

In [92]:
driver = webdriver.Chrome(options=options)
driver.get(url='')

In [93]:
df = pd.DataFrame()

In [94]:
for i in range(0, 775):
    xpath = f'/html/body/div[{i}]/div[2]/div[2]/div/div[2]/pre'
    try:
        element = driver.find_element("xpath", xpath)
        text = element.text.strip('[]')
        df[f'Column_{i}'] = [text]
    except:
        df[f'Column_{i}'] = [None]

In [95]:
df = df.dropna(axis=1, how='all')

In [96]:
cols_to_drop = [col for col in df.columns if df[col].apply(
    lambda x: str(x).startswith('(') or str(x).startswith('{') or 
              str(x).startswith('array') or str(x).startswith('ShapiroResult') or
            str(x).startswith('BartlettResult')
).any()]

df = df.drop(columns=cols_to_drop)

In [97]:
cols_to_drop_2 = [col for col in df.columns if not df[col].apply(lambda x: ',' in str(x)).any()]

df = df.drop(columns=cols_to_drop_2)

In [98]:
cols_to_drop_3 = [col for col in df.columns if df[col].apply(lambda x: 'alternative' in str(x)).any()]

df = df.drop(columns=cols_to_drop_3)

In [99]:
df

,Column_22,Column_34,Column_46,Column_58,Column_70,Column_82,Column_94,Column_107,Column_119,Column_131,...,Column_614,Column_626,Column_638,Column_650,Column_662,Column_674,Column_686,Column_698,Column_710,Column_722
0,"0.7704977998974709, 0.7582450444292549, 0.7685...","0.7673818779904307, 0.7633688268967874, 0.7689...","0.7642819762474367, 0.7762143284347232, 0.7656...","0.7713895890293916, 0.7704951298701299, 0.7670...","0.7704390592959671, 0.7702761876281614, 0.7689...","0.7699130639097745, 0.7738673744019139, 0.7730...","0.7609764823991797, 0.7709864149008886, 0.7616...","0.7751222872522215, 0.7748873248462065, 0.7604...","0.7745829417293233, 0.7802781100478469, 0.7706...","0.7712187072795625, 0.7733253588516746, 0.7705...",...,"0.7758805750170881, 0.7656410201640464, 0.7727...","0.7734375, 0.7744254101161996, 0.7676115003417...","0.7725483808954203, 0.781111158578264, 0.77624...","0.7636291545625427, 0.7686674961551605, 0.7603...","0.7682483018626111, 0.7620031079118249, 0.7661...","0.7598483958475735, 0.7645423039131921, 0.7573...","0.7561704331852357, 0.7522668532125769, 0.7532...","0.7496528964456596, 0.7462859919685578, 0.7468...","0.7289735346889952, 0.7289788747436774, 0.7367...","0.7036696855775801, 0.7057362867395761, 0.7064..."


In [100]:
all_data = []

for column in df.columns:
    column_data = list(map(float, df[column].iloc[0].split(', ')))
    all_data.append(column_data)

all_data_array = np.array(all_data).T
new_df = pd.DataFrame(all_data_array, columns=[f'ext_{i}' for i in range(len(df.columns))])

In [101]:
len_all_data = [len(sublist) for sublist in all_data]
sum(len_all_data) # 2000*59

118000

In [102]:
new_df.shape

(2000, 59)

In [103]:
new_df = new_df.astype(float)

In [104]:
new_df

,ext_0,ext_1,ext_2,ext_3,ext_4,ext_5,ext_6,ext_7,ext_8,ext_9,...,ext_49,ext_50,ext_51,ext_52,ext_53,ext_54,ext_55,ext_56,ext_57,ext_58
0,0.7704977998974709,0.7673818779904307,0.7642819762474367,0.7713895890293916,0.7704390592959671,0.7699130639097745,0.7609764823991797,0.7751222872522215,0.7745829417293233,0.7712187072795625,...,0.7758805750170881,0.7734375000000000,0.7725483808954203,0.7636291545625427,0.7682483018626111,0.7598483958475735,0.7561704331852357,0.7496528964456596,0.7289735346889952,0.7036696855775801
1,0.7582450444292549,0.7633688268967874,0.7762143284347232,0.7704951298701299,0.7702761876281614,0.7738673744019139,0.7709864149008886,0.7748873248462065,0.7802781100478469,0.7733253588516746,...,0.7656410201640464,0.7744254101161996,0.7811111585782640,0.7686674961551605,0.7620031079118249,0.7645423039131921,0.7522668532125769,0.7462859919685578,0.7289788747436774,0.7057362867395761
2,0.7685086295283663,0.7689758843130554,0.7656250000000000,0.7670427845181135,0.7689438439849625,0.7730583561175668,0.7616973897812713,0.7604211167122352,0.7706099410457962,0.7705485304169515,...,0.7727699931647301,0.7676115003417636,0.7762463687628162,0.7603743912337662,0.7661496603725222,0.7573759505297335,0.7532734535201639,0.7468013072453863,0.7367246240601505,0.7064812243677374
3,0.7715417805878333,0.7737846035543404,0.7701079759056733,0.7759126153451812,0.7735576512303486,0.7664473684210527,0.7665568395420369,0.7757150333219412,0.7710104451469582,0.7722573479152425,...,0.7709196642173617,0.7774558911483254,0.7684472188995215,0.7642993314251538,0.7663792827238551,0.7629162572624744,0.7485154647983597,0.7550330015379357,0.7303299085782639,0.7064251537935747
4,0.7678624829118250,0.7662124060150375,0.7728527640123035,0.7649094326725906,0.7653980476760082,0.7707648026315790,0.7696994617224882,0.7756162423103214,0.7753679297676008,0.7748098940533151,...,0.7769539260082023,0.7737365430622010,0.7701613764524948,0.7766188375768968,0.7619416972829801,0.7649855284518114,0.7523896744702666,0.7504405545112782,0.7333310193096377,0.7058964883800409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.7692642472658919,0.7655876196172249,0.7776748333902940,0.7769779562542720,0.7586802588858510,0.7762063183526999,0.7562985944976077,0.7703402682843472,0.7700652554682160,0.7697395121326042,...,0.7706353063055367,0.7808174555707450,0.7634302375256322,0.7680987803315105,0.7614824525803144,0.7633541417464117,0.7565228767942584,0.7503657937457279,0.7326368122009570,0.7032237910116198
1996,0.7712320574162679,0.7684685791182502,0.7757444036226930,0.7760808270676692,0.7744574504442925,0.7733120087149692,0.7689411739576213,0.7643433868762817,0.7720837961380724,0.7720544258373205,...,0.7749567455570745,0.7653820275119617,0.7676061602870814,0.7683631130382775,0.7613142408578265,0.7592716699419002,0.7555216165413532,0.7455917848598770,0.7354750512645249,0.7064251537935747
1997,0.7822272300068354,0.7700839456596036,0.7630110432330828,0.7758245044429254,0.7713949290840738,0.7617908407382092,0.7684338687628162,0.7769432458988380,0.7777549342105262,0.7731624871838689,...,0.7743853597060832,0.7721398667122351,0.7708449034518114,0.7641017494019139,0.7642405908236499,0.7626118741455913,0.7445157638414218,0.7459442284688996,0.7338036141490090,0.7043585526315788
1998,0.7689411739576214,0.7667998120300752,0.7758885850991114,0.7748098940533151,0.7795972530758716,0.7734107997265892,0.7685353298017772,0.7759740259740261,0.7766575529733424,0.7658305921052632,...,0.7767216336295284,0.7675928101503759,0.7672083262132603,0.7678584778708135,0.7660642194976077,0.7683818032296651,0.7556390977443610,0.7507342575187971,0.7361959586466166,0.7043585526315788


In [105]:
driver.close()
driver.quit()

In [106]:
print('AUC 신뢰 구간')
for column in new_df.columns:
    percentiles = np.percentile(new_df[column], [2.5, 97.5])
    rounded_percentiles = np.round(percentiles, 8)
    print(f"{column}: [{rounded_percentiles[0]} {rounded_percentiles[1]}]")

AUC 신뢰 구간
ext_0: [0.76042646 0.77867963]
ext_1: [0.75836373 0.77687703]
ext_2: [0.75940998 0.777862]
ext_3: [0.76096307 0.77827372]
ext_4: [0.76207887 0.77985391]
ext_5: [0.76219889 0.77980672]
ext_6: [0.75949195 0.77746931]
ext_7: [0.7616755 0.77873777]
ext_8: [0.76200157 0.77978349]
ext_9: [0.7618654 0.77940735]
ext_10: [0.76211445 0.77938212]
ext_11: [0.75942426 0.77773644]
ext_12: [0.76040897 0.77851122]
ext_13: [0.76288035 0.78016971]
ext_14: [0.75893618 0.77822553]
ext_15: [0.7627082 0.77898442]
ext_16: [0.76252103 0.7792382]
ext_17: [0.75940043 0.77723808]
ext_18: [0.76260226 0.77943071]
ext_19: [0.76087299 0.77814075]
ext_20: [0.75862672 0.77685914]
ext_21: [0.75967318 0.77790179]
ext_22: [0.76120564 0.77817286]
ext_23: [0.75964581 0.77733574]
ext_24: [0.76071676 0.77826598]
ext_25: [0.76090953 0.77895124]
ext_26: [0.7596966 0.77671416]
ext_27: [0.76157944 0.77870613]
ext_28: [0.76074613 0.77755308]
ext_29: [0.76166535 0.77800111]
ext_30: [0.76160995 0.77939233]
ext_31: [0.7615

In [107]:
print('AUC 신뢰 구간')
for column in new_df.columns:
    percentiles = np.percentile(new_df[column], [2.5, 97.5])
    rounded_percentiles = np.round(percentiles, 2)
    print(f"{column}: [{rounded_percentiles[0]} {rounded_percentiles[1]}]")

AUC 신뢰 구간
ext_0: [0.76 0.78]
ext_1: [0.76 0.78]
ext_2: [0.76 0.78]
ext_3: [0.76 0.78]
ext_4: [0.76 0.78]
ext_5: [0.76 0.78]
ext_6: [0.76 0.78]
ext_7: [0.76 0.78]
ext_8: [0.76 0.78]
ext_9: [0.76 0.78]
ext_10: [0.76 0.78]
ext_11: [0.76 0.78]
ext_12: [0.76 0.78]
ext_13: [0.76 0.78]
ext_14: [0.76 0.78]
ext_15: [0.76 0.78]
ext_16: [0.76 0.78]
ext_17: [0.76 0.78]
ext_18: [0.76 0.78]
ext_19: [0.76 0.78]
ext_20: [0.76 0.78]
ext_21: [0.76 0.78]
ext_22: [0.76 0.78]
ext_23: [0.76 0.78]
ext_24: [0.76 0.78]
ext_25: [0.76 0.78]
ext_26: [0.76 0.78]
ext_27: [0.76 0.78]
ext_28: [0.76 0.78]
ext_29: [0.76 0.78]
ext_30: [0.76 0.78]
ext_31: [0.76 0.78]
ext_32: [0.76 0.78]
ext_33: [0.76 0.78]
ext_34: [0.76 0.78]
ext_35: [0.76 0.78]
ext_36: [0.76 0.78]
ext_37: [0.76 0.78]
ext_38: [0.77 0.78]
ext_39: [0.76 0.78]
ext_40: [0.76 0.78]
ext_41: [0.76 0.78]
ext_42: [0.76 0.78]
ext_43: [0.77 0.78]
ext_44: [0.77 0.78]
ext_45: [0.77 0.78]
ext_46: [0.77 0.78]
ext_47: [0.77 0.78]
ext_48: [0.76 0.78]
ext_49: [0.76 0.78]


In [108]:
i = 0
while i < 58:  
    base_column = f'ext_{i}'
    continue_search = True  

    for j in range(i + 1, 59):
        if not continue_search:
            break  
        compare_column = f'ext_{j}'
        base_stats = sms.DescrStatsW(new_df[base_column])
        compare_stats = sms.DescrStatsW(new_df[compare_column])
        cm = sms.CompareMeans(base_stats, compare_stats)
        confint = cm.tconfint_diff(usevar='unequal')

        print(f'{base_column} and {compare_column}:')
        print(confint)
        print(cm.summary())
        print('-------------------------------------------')
        
        if confint[0] <= 0 <= confint[1] or (confint[0] < 0 and confint[1] < 0):
            continue_search = False  
        if compare_column == 'ext_58':
            break
    if compare_column == 'ext_58' and continue_search:
        break
    i += 1  

ext_0 and ext_1:
(0.001741141210486655, 0.0023210730965883297)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1      0.0020      0.000     13.733      0.000       0.002       0.002
-------------------------------------------
ext_0 and ext_2:
(0.0006365395606425619, 0.0012187369473889607)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1      0.0009      0.000      6.248      0.000       0.001       0.001
-------------------------------------------
ext_0 and ext_3:
(-0.0002417838340549143, 0.0003231796175477801)
                          Test for equality of means                          
         

In [110]:
### ext = 47

In [111]:
del new_df

### rf

In [132]:
driver = webdriver.Chrome(options=options)
driver.get(url='.html')

In [133]:
df = pd.DataFrame()

In [134]:
for i in range(0, 775):
    xpath = f'/html/body/div[{i}]/div[2]/div[2]/div/div[2]/pre'
    try:
        element = driver.find_element("xpath", xpath)
        text = element.text.strip('[]')
        df[f'Column_{i}'] = [text]
    except:
        df[f'Column_{i}'] = [None]

df = df.dropna(axis=1, how='all')

In [135]:
cols_to_drop = [col for col in df.columns if df[col].apply(
    lambda x: str(x).startswith('(') or str(x).startswith('{') or 
              str(x).startswith('array') or str(x).startswith('ShapiroResult') or
            str(x).startswith('BartlettResult')
).any()]

df = df.drop(columns=cols_to_drop)

In [136]:
cols_to_drop_2 = [col for col in df.columns if not df[col].apply(lambda x: ',' in str(x)).any()]

df = df.drop(columns=cols_to_drop_2)

In [137]:
cols_to_drop_3 = [col for col in df.columns if df[col].apply(lambda x: 'alternative' in str(x)).any()]

df = df.drop(columns=cols_to_drop_3)

In [138]:
pd.set_option('display.max_columns', None)

In [139]:
df

,Column_23,Column_35,Column_47,Column_59,Column_71,Column_83,Column_95,Column_107,Column_120,Column_132,Column_144,Column_156,Column_168,Column_180,Column_192,Column_204,Column_216,Column_228,Column_240,Column_252,Column_264,Column_276,Column_288,Column_300,Column_312,Column_324,Column_336,Column_348,Column_360,Column_372,Column_384,Column_396,Column_408,Column_420,Column_432,Column_444,Column_456,Column_468,Column_480,Column_492,Column_504,Column_505,Column_517,Column_531,Column_544,Column_557,Column_570,Column_583,Column_596,Column_609,Column_622,Column_635,Column_648,Column_661,Column_674,Column_687,Column_700,Column_713,Column_726,Column_739
0,"0.7781314080656185, 0.7736083817498292, 0.7778...","0.7659267130895421, 0.7703055579289132, 0.7770...","0.7690159347231716, 0.7753465695488722, 0.7753...","0.7686768412508544, 0.7694725093984962, 0.7715...","0.7759633458646618, 0.7719850051264525, 0.7705...","0.771637901572112, 0.7673631877990431, 0.77677...","0.7670214242993849, 0.773472210355434, 0.76881...","0.7785559424128503, 0.7781180579289132, 0.7641...","0.7747538234791523, 0.7773971505468218, 0.7786...","0.7749060150375939, 0.7753999700956937, 0.7751...","0.7749247052289815, 0.7757283834586466, 0.7759...","0.7755975521189337, 0.7762970992822966, 0.7744...","0.776409240430622, 0.773336038961039, 0.775853...","0.7835195232399179, 0.7682816772043746, 0.7699...","0.7738219839371157, 0.7750849068694464, 0.7756...","0.7764626409774436, 0.7675874700956937, 0.7778...","0.7742865686944636, 0.7672857570061518, 0.7576...","0.7789858168147641, 0.7775012816131237, 0.7797...","0.7751116071428571, 0.7722947282980178, 0.7783...","0.7795999231032126, 0.7681775461380724, 0.7783...","0.7770046565276829, 0.7806358937115516, 0.7789...","0.7738059637730691, 0.777341079972659, 0.77602...","0.7717633928571428, 0.7762009782980177, 0.7730...","0.7698569933356116, 0.776043446684894, 0.77912...","0.7776881835269992, 0.7728741242310322, 0.7826...","0.7720544258373205, 0.7753679297676008, 0.7756...","0.7756295924470267, 0.7725003204032809, 0.7768...","0.7765320616883117, 0.7720731160287081, 0.7757...","0.778078007518797, 0.7722279776144907, 0.77999...","0.7768257646958304, 0.781650504101162, 0.77824...","0.7754880809979494, 0.7832979109706083, 0.7796...","0.7788896958304854, 0.7771942284688995, 0.7727...","0.7772769993164731, 0.7780459671907038, 0.7774...","0.7811031484962406, 0.7792982100136705, 0.7858...","0.7809269266917293, 0.7761742780246069, 0.7756...","0.780216699419002, 0.7788896958304853, 0.78079...","0.774102336807929, 0.7860373590225564, 0.78253...","0.7832712106971975, 0.7803929212235134, 0.7791...","0.7789270762132604, 0.7778083347573479, 0.7849...","0.779744104579631, 0.7793756408065617, 0.77985...","0.7805290926179084, 0.7781207279562542, 0.7798...",----------------------------------------------...,"0.7847637559808613, 0.7800404776144909, 0.7810...","0.7723107484620643, 0.7757363935406698, 0.7811...","0.7757283834586466, 0.7772476290157212, 0.7798...","0.7887607869104579, 0.7777976546479837, 0.7828...","0.7799043062200957, 0.7727472979323309, 0.7730...","0.784953327922078, 0.7812606801093644, 0.78445...","0.7722039473684211, 0.773103746582365, 0.77353...","0.7808067754613807, 0.7725363657723855, 0.7788...","0.7733800944121667, 0.7813527960526316, 0.7791...","0.7779925666438825, 0.7757937991285031, 0.7786...","0.7674286034688996, 0.7709717297505125, 0.7730...","0.7661616754955571, 0.778562617481203, 0.76867...","0.7722199675324677, 0.7651737653793573, 0.7674...","0.7612581702836637, 0.7626746197881067, 0.7557...","0.7539289452323991, 0.7570795774948735, 0.7523...","0.7440805493848258, 0.7462339264354068, 0.7368...","0.7292966079972659, 0.7337875939849624, 0.7382...","0.7058243976418317, 0.7064812243677374, 0.7057..."


In [140]:
df = df.drop('Column_505', axis=1)

In [141]:
all_data = []

for column in df.columns:
    column_data = list(map(float, df[column].iloc[0].split(', ')))
    all_data.append(column_data)

all_data_array = np.array(all_data).T
new_df = pd.DataFrame(all_data_array, columns=[f'rf_{i}' for i in range(len(df.columns))])

In [142]:
len_all_data = []
len_all_data = [len(sublist) for sublist in all_data]
sum(len_all_data) # 2000 * 59

118000

In [143]:
new_df.shape

(2000, 59)

In [144]:
new_df = new_df.astype(float)

In [145]:
new_df

,rf_0,rf_1,rf_2,rf_3,rf_4,rf_5,rf_6,rf_7,rf_8,rf_9,rf_10,rf_11,rf_12,rf_13,rf_14,rf_15,rf_16,rf_17,rf_18,rf_19,rf_20,rf_21,rf_22,rf_23,rf_24,rf_25,rf_26,rf_27,rf_28,rf_29,rf_30,rf_31,rf_32,rf_33,rf_34,rf_35,rf_36,rf_37,rf_38,rf_39,rf_40,rf_41,rf_42,rf_43,rf_44,rf_45,rf_46,rf_47,rf_48,rf_49,rf_50,rf_51,rf_52,rf_53,rf_54,rf_55,rf_56,rf_57,rf_58
0,0.7781314080656185,0.7659267130895421,0.7690159347231716,0.7686768412508544,0.7759633458646618,0.7716379015721120,0.7670214242993849,0.7785559424128503,0.7747538234791523,0.7749060150375939,0.7749247052289815,0.7755975521189337,0.7764092404306220,0.7835195232399179,0.7738219839371157,0.7764626409774436,0.7742865686944636,0.7789858168147641,0.7751116071428571,0.7795999231032126,0.7770046565276829,0.7738059637730691,0.7717633928571428,0.7698569933356116,0.7776881835269992,0.7720544258373205,0.7756295924470267,0.7765320616883117,0.7780780075187970,0.7768257646958304,0.7754880809979494,0.7788896958304854,0.7772769993164731,0.7811031484962406,0.7809269266917293,0.7802166994190020,0.7741023368079289,0.7832712106971975,0.7789270762132604,0.7797441045796309,0.7805290926179084,0.7847637559808613,0.7723107484620643,0.7757283834586466,0.7887607869104579,0.7799043062200957,0.7849533279220779,0.7722039473684211,0.7808067754613807,0.7733800944121667,0.7779925666438825,0.7674286034688996,0.7661616754955571,0.7722199675324677,0.7612581702836637,0.7539289452323991,0.7440805493848258,0.7292966079972659,0.7058243976418317
1,0.7736083817498292,0.7703055579289132,0.7753465695488722,0.7694725093984962,0.7719850051264525,0.7673631877990431,0.7734722103554340,0.7781180579289132,0.7773971505468218,0.7753999700956937,0.7757283834586466,0.7762970992822966,0.7733360389610390,0.7682816772043746,0.7750849068694464,0.7675874700956937,0.7672857570061518,0.7775012816131237,0.7722947282980178,0.7681775461380724,0.7806358937115516,0.7773410799726590,0.7762009782980177,0.7760434466848940,0.7728741242310322,0.7753679297676008,0.7725003204032809,0.7720731160287081,0.7722279776144907,0.7816505041011620,0.7832979109706083,0.7771942284688995,0.7780459671907038,0.7792982100136705,0.7761742780246069,0.7788896958304853,0.7860373590225564,0.7803929212235134,0.7778083347573479,0.7793756408065617,0.7781207279562542,0.7800404776144909,0.7757363935406698,0.7772476290157212,0.7777976546479837,0.7727472979323309,0.7812606801093644,0.7731037465823650,0.7725363657723855,0.7813527960526316,0.7757937991285031,0.7709717297505125,0.7785626174812030,0.7651737653793573,0.7626746197881067,0.7570795774948735,0.7462339264354068,0.7337875939849624,0.7064812243677374
2,0.7778003246753247,0.7770500469924813,0.7753492395762133,0.7715417805878332,0.7705218301435407,0.7767750341763500,0.7688103426179085,0.7641511449077238,0.7786841037252221,0.7751436474709501,0.7759473257006152,0.7744574504442925,0.7758538747436774,0.7699931647300069,0.7756002221462748,0.7778724154135339,0.7576042378673957,0.7797441045796309,0.7783102998974709,0.7783770505809979,0.7789991669514696,0.7760274265208474,0.7730957365003418,0.7791273282638413,0.7826971548188653,0.7756376025290499,0.7768871753246753,0.7757871240601504,0.7799977571770336,0.7782462192412851,0.7796960440874915,0.7727486329460014,0.7774959415584415,0.7858958475734793,0.7756776529391660,0.7807960953520164,0.7825396232057418,0.7791086380724540,0.7849292976760082,0.7798509056732741,0.7798856160287081,0.7810390678400547,0.7811245087149693,0.7798669258373205,0.7828279861585783,0.7730877264183185,0.7844567028366369,0.7735309509569378,0.7788990409261791,0.7791887388926863,0.7786894437799043,0.7730022855434039,0.7686768412508543,0.7674379485645932,0.7557899542891320,0.7523282638414217,0.7368621304682160,0.7382251794258374,0.7057683270676691
3,0.7762009782980177,0.7786600734791524,0.7699050538277512,0.7799336765208476,0.7721291866028708,0.7681027853725222,0.7720143754272044,0.7733840994531784,0.7659614234449761,0.7737552332535886,0.7685860603212576,0.7718621838687628,0.7719636449077238,0.7703776486671223,0.7682816772043746

In [146]:
driver.close()
driver.quit()

In [147]:
print('AUC 신뢰 구간')
for column in new_df.columns:
    percentiles = np.percentile(new_df[column], [2.5, 97.5])
    rounded_percentiles = np.round(percentiles, 8)
    print(f"{column}: [{rounded_percentiles[0]} {rounded_percentiles[1]}]")

AUC 신뢰 구간
rf_0: [0.76491951 0.78174683]
rf_1: [0.76491457 0.78314118]
rf_2: [0.76422824 0.7829181]
rf_3: [0.76330695 0.78263488]
rf_4: [0.76405449 0.78249103]
rf_5: [0.76400449 0.78202618]
rf_6: [0.76409167 0.78218758]
rf_7: [0.7637483 0.78199113]
rf_8: [0.76141136 0.78076986]
rf_9: [0.76309615 0.7822594]
rf_10: [0.76485683 0.78249283]
rf_11: [0.76447949 0.78304419]
rf_12: [0.76678559 0.78400854]
rf_13: [0.76620633 0.78336553]
rf_14: [0.76261281 0.78162167]
rf_15: [0.76716968 0.78415766]
rf_16: [0.76301625 0.78277746]
rf_17: [0.7658332 0.78243042]
rf_18: [0.76583046 0.78197698]
rf_19: [0.76506396 0.78232262]
rf_20: [0.76357902 0.78193126]
rf_21: [0.76690868 0.78273527]
rf_22: [0.76691109 0.78358681]
rf_23: [0.76554724 0.78317335]
rf_24: [0.7636939 0.78087366]
rf_25: [0.76545605 0.78356578]
rf_26: [0.76736766 0.78473852]
rf_27: [0.76471939 0.78318977]
rf_28: [0.76997875 0.7859146]
rf_29: [0.77038299 0.78638713]
rf_30: [0.77011799 0.7869975]
rf_31: [0.76951189 0.78613388]
rf_32: [0.76797

In [148]:
print('AUC 신뢰 구간')
for column in new_df.columns:
    percentiles = np.percentile(new_df[column], [2.5, 97.5])
    rounded_percentiles = np.round(percentiles, 2)
    print(f"{column}: [{rounded_percentiles[0]} {rounded_percentiles[1]}]")

AUC 신뢰 구간
rf_0: [0.76 0.78]
rf_1: [0.76 0.78]
rf_2: [0.76 0.78]
rf_3: [0.76 0.78]
rf_4: [0.76 0.78]
rf_5: [0.76 0.78]
rf_6: [0.76 0.78]
rf_7: [0.76 0.78]
rf_8: [0.76 0.78]
rf_9: [0.76 0.78]
rf_10: [0.76 0.78]
rf_11: [0.76 0.78]
rf_12: [0.77 0.78]
rf_13: [0.77 0.78]
rf_14: [0.76 0.78]
rf_15: [0.77 0.78]
rf_16: [0.76 0.78]
rf_17: [0.77 0.78]
rf_18: [0.77 0.78]
rf_19: [0.77 0.78]
rf_20: [0.76 0.78]
rf_21: [0.77 0.78]
rf_22: [0.77 0.78]
rf_23: [0.77 0.78]
rf_24: [0.76 0.78]
rf_25: [0.77 0.78]
rf_26: [0.77 0.78]
rf_27: [0.76 0.78]
rf_28: [0.77 0.79]
rf_29: [0.77 0.79]
rf_30: [0.77 0.79]
rf_31: [0.77 0.79]
rf_32: [0.77 0.79]
rf_33: [0.77 0.79]
rf_34: [0.77 0.79]
rf_35: [0.77 0.79]
rf_36: [0.77 0.79]
rf_37: [0.77 0.79]
rf_38: [0.77 0.79]
rf_39: [0.77 0.79]
rf_40: [0.77 0.78]
rf_41: [0.77 0.79]
rf_42: [0.77 0.79]
rf_43: [0.77 0.79]
rf_44: [0.77 0.79]
rf_45: [0.77 0.78]
rf_46: [0.77 0.79]
rf_47: [0.77 0.79]
rf_48: [0.77 0.79]
rf_49: [0.77 0.78]
rf_50: [0.76 0.78]
rf_51: [0.76 0.78]
rf_52: [0.76

In [149]:
i = 0
while i < 58:  
    base_column = f'rf_{i}'
    continue_search = True  

    for j in range(i + 1, 59):
        if not continue_search:
            break  
        compare_column = f'rf_{j}'
        base_stats = sms.DescrStatsW(new_df[base_column])
        compare_stats = sms.DescrStatsW(new_df[compare_column])
        cm = sms.CompareMeans(base_stats, compare_stats)
        confint = cm.tconfint_diff(usevar='unequal')

        print(f'{base_column} and {compare_column}:')
        print(confint)
        print(cm.summary())
        print('-------------------------------------------')
        
        if confint[0] <= 0 <= confint[1] or (confint[0] < 0 and confint[1] < 0):
            continue_search = False  
        if compare_column == 'rf_58':
            break
    if compare_column == 'rf_58' and continue_search:
        break
    i += 1   

rf_0 and rf_1:
(-0.0007235480789982441, -0.0001660222602022076)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1     -0.0004      0.000     -3.128      0.002      -0.001      -0.000
-------------------------------------------
rf_1 and rf_2:
(-4.929006548180169e-05, 0.0005293476312710841)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1      0.0002      0.000      1.627      0.104   -4.93e-05       0.001
-------------------------------------------
rf_2 and rf_3:
(0.0005403584933051805, 0.0011332306642480431)
                          Test for equality of means                          
            

In [150]:
## rf = 43

In [151]:
del new_df

### dt

In [166]:
driver = webdriver.Chrome(options=options)
driver.get(url='.html')

In [167]:
df = pd.DataFrame()

In [168]:
for i in range(0, 800):
    xpath = f'/html/body/div[{i}]/div[2]/div[2]/div/div[2]/pre'
    try:
        element = driver.find_element("xpath", xpath)
        text = element.text.strip('[]')
        df[f'Column_{i}'] = [text]
    except:
        df[f'Column_{i}'] = [None]

df = df.dropna(axis=1, how='all')

In [169]:
cols_to_drop = [col for col in df.columns if df[col].apply(
    lambda x: str(x).startswith('(') or str(x).startswith('{') or 
              str(x).startswith('array') or str(x).startswith('ShapiroResult') or
            str(x).startswith('BartlettResult')
).any()]

df = df.drop(columns=cols_to_drop)

In [170]:
cols_to_drop_2 = [col for col in df.columns if not df[col].apply(lambda x: ',' in str(x)).any()]

df = df.drop(columns=cols_to_drop_2)

In [171]:
cols_to_drop_3 = [col for col in df.columns if df[col].apply(lambda x: 'alternative' in str(x)).any()]

df = df.drop(columns=cols_to_drop_3)

In [172]:
df

,Column_28,Column_44,Column_58,Column_71,Column_84,Column_96,Column_109,Column_122,Column_135,Column_148,Column_162,Column_175,Column_189,Column_202,Column_215,Column_228,Column_241,Column_254,Column_267,Column_280,Column_292,Column_305,Column_318,Column_331,Column_344,Column_357,Column_370,Column_383,Column_396,Column_409,Column_422,Column_435,Column_448,Column_461,Column_474,Column_487,Column_500,Column_513,Column_526,Column_539,Column_552,Column_565,Column_578,Column_591,Column_604,Column_617,Column_630,Column_643,Column_656,Column_669,Column_682,Column_695,Column_708,Column_721,Column_734,Column_747,Column_760,Column_773,Column_786
0,"0.7167861948906358, 0.7073770185406698, 0.7172...","0.7020115985987696, 0.7060339947881066, 0.7122...","0.7107786333732057, 0.7179756920710869, 0.7090...","0.7055680750170882, 0.7141976033834586, 0.6964...","0.7122097680280246, 0.7113153088687627, 0.7283...","0.7161787636705399, 0.7115996667805878, 0.6876...","0.6980919984620643, 0.6936704331852358, 0.7146...","0.7054799641148324, 0.6914943609022556, 0.6770...","0.7026684253246754, 0.7271338858509911, 0.7047...","0.7068283279220778, 0.7091899671052632, 0.6993...","0.7217911611414901, 0.7237776614832536, 0.7093...","0.7171786889097744, 0.7052730369958988, 0.7100...","0.698365676264525, 0.6696602123205742, 0.69202...","0.7235373590225564, 0.7333497095010253, 0.6884...","0.708227422248804, 0.7077414772727273, 0.70165...","0.7119200700615174, 0.7148036995898839, 0.7058...","0.7029821535372522, 0.6983923765379356, 0.6968...","0.7282326021018456, 0.7080992609364319, 0.7103...","0.7024628332194123, 0.689348993933698, 0.72749...","0.7104141746411485, 0.7395975734791524, 0.7179...","0.6946169578776487, 0.7169250363123718, 0.7185...","0.7262821471291865, 0.7170198222829802, 0.7142...","0.7236615152939166, 0.7214480626281614, 0.7117...","0.7186391938653451, 0.7152202238550923, 0.7129...","0.722650909945318, 0.7222517408578263, 0.71494...","0.715014631749829, 0.6944527511961722, 0.71403...","0.7122818587662338, 0.7184883373205742, 0.7293...","0.7137143284347232, 0.7080939208817498, 0.6974...","0.7088815789473685, 0.6917092981032126, 0.7224...","0.6858258928571428, 0.7001479195146957, 0.7155...","0.6985645933014354, 0.715441836124402, 0.69015...","0.7139466208133971, 0.708615911226931, 0.69936...","0.710366114149009, 0.700074493762816, 0.690478...","0.7104435449419002, 0.6860381600307588, 0.6711...","0.7035174940191388, 0.7098561389268625, 0.6901...","0.7021971654989747, 0.7123753097231716, 0.6960...","0.7067121817327409, 0.6992761555878334, 0.6989...","0.714858435150376, 0.7125034710355433, 0.71451...","0.695013456937799, 0.6682811431989064, 0.67762...","0.6913862247949418, 0.6736252029220778, 0.6824...","0.6970613679084074, 0.6827073009227616, 0.7082...","0.6986273389439508, 0.6842425666438825, 0.6894...","0.6819677033492823, 0.7238097018113465, 0.6923...","0.6899871304682159, 0.7114127648667122, 0.6894...","0.7000104131066304, 0.6972068843984962, 0.6902...","0.7115502712747779, 0.7227056455058098, 0.7145...","0.7049459586466166, 0.717029167378674, 0.69992...","0.7092487077067668, 0.7085104451469582, 0.7147...","0.7231715652768285, 0.7206083390293916, 0.7111...","0.7363027597402598, 0.719966197453862, 0.71498...","0.7356913234791524, 0.716782189849624, 0.72378...","0.7018447218899523, 0.7061301157723855, 0.7382...","0.6958598556049215, 0.7078909988038277, 0.7060...","0.7069204438653451, 0.728468899521531, 0.72576...","0.7283634334415583, 0.7157395441729323, 0.7082...","0.7191651892515379, 0.7349143455228981, 0.7137...","0.7154765464798358, 0.7127864939336979, 0.7215...","0.7192239298530416, 0.714702238550923, 0.70772...","0.7057362867395761, 0.7057362867395761, 0.7064..."


In [173]:
all_data = []

for column in df.columns:
    column_data = list(map(float, df[column].iloc[0].split(', ')))
    all_data.append(column_data)

all_data_array = np.array(all_data).T
new_df = pd.DataFrame(all_data_array, columns=[f'dt_{i}' for i in range(len(df.columns))])

In [174]:
len_all_data = []
len_all_data = [len(sublist) for sublist in all_data]
sum(len_all_data) # 2000 * 59

118000

In [175]:
new_df.shape

(2000, 59)

In [176]:
new_df = new_df.astype(float)

In [177]:
new_df

,dt_0,dt_1,dt_2,dt_3,dt_4,dt_5,dt_6,dt_7,dt_8,dt_9,dt_10,dt_11,dt_12,dt_13,dt_14,dt_15,dt_16,dt_17,dt_18,dt_19,dt_20,dt_21,dt_22,dt_23,dt_24,dt_25,dt_26,dt_27,dt_28,dt_29,dt_30,dt_31,dt_32,dt_33,dt_34,dt_35,dt_36,dt_37,dt_38,dt_39,dt_40,dt_41,dt_42,dt_43,dt_44,dt_45,dt_46,dt_47,dt_48,dt_49,dt_50,dt_51,dt_52,dt_53,dt_54,dt_55,dt_56,dt_57,dt_58
0,0.7167861948906358,0.7020115985987696,0.7107786333732057,0.7055680750170882,0.7122097680280246,0.7161787636705399,0.6980919984620643,0.7054799641148324,0.7026684253246754,0.7068283279220778,0.7217911611414901,0.7171786889097744,0.6983656762645249,0.7235373590225564,0.7082274222488040,0.7119200700615174,0.7029821535372522,0.7282326021018456,0.7024628332194123,0.7104141746411485,0.6946169578776487,0.7262821471291865,0.7236615152939166,0.7186391938653451,0.7226509099453180,0.7150146317498290,0.7122818587662338,0.7137143284347232,0.7088815789473685,0.6858258928571428,0.6985645933014354,0.7139466208133971,0.7103661141490090,0.7104435449419002,0.7035174940191388,0.7021971654989747,0.7067121817327409,0.7148584351503759,0.6950134569377990,0.6913862247949418,0.6970613679084074,0.6986273389439508,0.6819677033492823,0.6899871304682159,0.7000104131066304,0.7115502712747779,0.7049459586466166,0.7092487077067668,0.7231715652768285,0.7363027597402598,0.7356913234791524,0.7018447218899523,0.6958598556049215,0.7069204438653451,0.7283634334415583,0.7191651892515379,0.7154765464798358,0.7192239298530416,0.7057362867395761
1,0.7073770185406698,0.7060339947881066,0.7179756920710869,0.7141976033834586,0.7113153088687627,0.7115996667805878,0.6936704331852358,0.6914943609022556,0.7271338858509911,0.7091899671052632,0.7237776614832536,0.7052730369958988,0.6696602123205742,0.7333497095010253,0.7077414772727273,0.7148036995898839,0.6983923765379356,0.7080992609364319,0.6893489939336980,0.7395975734791524,0.7169250363123718,0.7170198222829802,0.7214480626281614,0.7152202238550923,0.7222517408578263,0.6944527511961722,0.7184883373205742,0.7080939208817498,0.6917092981032126,0.7001479195146957,0.7154418361244020,0.7086159112269310,0.7000744937628161,0.6860381600307588,0.7098561389268625,0.7123753097231716,0.6992761555878334,0.7125034710355433,0.6682811431989064,0.6736252029220778,0.6827073009227616,0.6842425666438825,0.7238097018113465,0.7114127648667122,0.6972068843984962,0.7227056455058098,0.7170291673786739,0.7085104451469582,0.7206083390293916,0.7199661974538620,0.7167821898496241,0.7061301157723855,0.7078909988038277,0.7284688995215310,0.7157395441729323,0.7349143455228981,0.7127864939336979,0.7147022385509230,0.7057362867395761
2,0.7172975051264525,0.7122604985475052,0.7090097402597404,0.6964793019480521,0.7283193779904307,0.6876922419685578,0.7146127926349967,0.6770067925495554,0.7047470416097061,0.6993522513670540,0.7093755340054684,0.7100096654989746,0.6920203562884484,0.6884772300068353,0.7016551499487355,0.7058510979152428,0.6968851461038962,0.7103060385338347,0.7274983445830486,0.7179236265379357,0.7185564230177717,0.7142723641490090,0.7117905737354749,0.7129707258202325,0.7149438760252905,0.7140320616883116,0.7293593536397812,0.6974418468045114,0.7224626730177717,0.7155246069719754,0.6901553421907041,0.6993656015037595,0.6904784154989747,0.6711140422077922,0.6901166267942584,0.6960187222317157,0.6989544172932330,0.7145126666097060,0.6776289089200274,0.6824362931476418,0.7082861628503077,0.6894571300410116,0.6923701298701297,0.6894771552460697,0.6902541332023241,0.7145473769651401,0.6999263072453862,0.7147823393711551,0.7111137218045113,0.7149852614490771,0.7237883415926178,0.7382171693438140,0.7060767152255638,0.7257668318523581,0.7082968429596719,0.7137236735304169,0.7215561987354751,0.7077214520676692,0.7064812243677374
3,0.7121483573991798,0.7268335077751196,0.7017018754272044,0.7200609834244702,0.7142990644224196,0.7162308292036910,0.7134566707963089,0.6865040798017772,0.7006512196684894,0.7156527682843472,0.7165886128673957,0.6920350414388241,0.6813522620471634,0.7180744830827068,0.6817848064764185

In [178]:
driver.close()
driver.quit()

In [179]:
print('AUC 신뢰 구간')
for column in new_df.columns:
    percentiles = np.percentile(new_df[column], [2.5, 97.5])
    rounded_percentiles = np.round(percentiles, 8)
    print(f"{column}: [{rounded_percentiles[0]} {rounded_percentiles[1]}]")

AUC 신뢰 구간
dt_0: [0.68622049 0.73171438]
dt_1: [0.6861775 0.73303234]
dt_2: [0.68644464 0.73254022]
dt_3: [0.68762886 0.73284591]
dt_4: [0.68727882 0.73190045]
dt_5: [0.68723844 0.73294273]
dt_6: [0.6868912 0.73239708]
dt_7: [0.66179531 0.71495075]
dt_8: [0.68822528 0.73293412]
dt_9: [0.68775098 0.73332201]
dt_10: [0.68857699 0.73234668]
dt_11: [0.68899652 0.73363757]
dt_12: [0.66159122 0.71682451]
dt_13: [0.68629725 0.73280673]
dt_14: [0.68782574 0.73165624]
dt_15: [0.6880044 0.73288719]
dt_16: [0.68839673 0.73256889]
dt_17: [0.68822752 0.73447256]
dt_18: [0.6872643 0.73259613]
dt_19: [0.68790327 0.73479089]
dt_20: [0.68873082 0.7333942]
dt_21: [0.68984378 0.73334781]
dt_22: [0.68922417 0.73280836]
dt_23: [0.6885368 0.73316912]
dt_24: [0.6880055 0.73419738]
dt_25: [0.68872684 0.73344109]
dt_26: [0.68930284 0.73356451]
dt_27: [0.67854032 0.72583942]
dt_28: [0.67720401 0.72754394]
dt_29: [0.67845221 0.72747208]
dt_30: [0.6772511 0.72583772]
dt_31: [0.67753482 0.72733167]
dt_32: [0.677879

In [180]:
print('AUC 신뢰 구간')
for column in new_df.columns:
    percentiles = np.percentile(new_df[column], [2.5, 97.5])
    rounded_percentiles = np.round(percentiles, 2)
    print(f"{column}: [{rounded_percentiles[0]} {rounded_percentiles[1]}]")

AUC 신뢰 구간
dt_0: [0.69 0.73]
dt_1: [0.69 0.73]
dt_2: [0.69 0.73]
dt_3: [0.69 0.73]
dt_4: [0.69 0.73]
dt_5: [0.69 0.73]
dt_6: [0.69 0.73]
dt_7: [0.66 0.71]
dt_8: [0.69 0.73]
dt_9: [0.69 0.73]
dt_10: [0.69 0.73]
dt_11: [0.69 0.73]
dt_12: [0.66 0.72]
dt_13: [0.69 0.73]
dt_14: [0.69 0.73]
dt_15: [0.69 0.73]
dt_16: [0.69 0.73]
dt_17: [0.69 0.73]
dt_18: [0.69 0.73]
dt_19: [0.69 0.73]
dt_20: [0.69 0.73]
dt_21: [0.69 0.73]
dt_22: [0.69 0.73]
dt_23: [0.69 0.73]
dt_24: [0.69 0.73]
dt_25: [0.69 0.73]
dt_26: [0.69 0.73]
dt_27: [0.68 0.73]
dt_28: [0.68 0.73]
dt_29: [0.68 0.73]
dt_30: [0.68 0.73]
dt_31: [0.68 0.73]
dt_32: [0.68 0.73]
dt_33: [0.66 0.72]
dt_34: [0.68 0.73]
dt_35: [0.68 0.73]
dt_36: [0.68 0.73]
dt_37: [0.69 0.74]
dt_38: [0.66 0.71]
dt_39: [0.66 0.71]
dt_40: [0.67 0.72]
dt_41: [0.67 0.72]
dt_42: [0.67 0.72]
dt_43: [0.68 0.72]
dt_44: [0.67 0.73]
dt_45: [0.69 0.73]
dt_46: [0.69 0.73]
dt_47: [0.69 0.73]
dt_48: [0.69 0.73]
dt_49: [0.69 0.74]
dt_50: [0.69 0.74]
dt_51: [0.69 0.73]
dt_52: [0.69

In [181]:
i = 0
while i < 58:  
    base_column = f'dt_{i}'
    continue_search = True  

    for j in range(i + 1, 59):
        if not continue_search:
            break  
        compare_column = f'dt_{j}'
        base_stats = sms.DescrStatsW(new_df[base_column])
        compare_stats = sms.DescrStatsW(new_df[compare_column])
        cm = sms.CompareMeans(base_stats, compare_stats)
        confint = cm.tconfint_diff(usevar='unequal')

        print(f'{base_column} and {compare_column}:')
        print(confint)
        print(cm.summary())
        print('-------------------------------------------')
        
        if confint[0] <= 0 <= confint[1] or (confint[0] < 0 and confint[1] < 0):
            continue_search = False  
        if compare_column == 'dt_58':
            break
    if compare_column == 'dt_58' and continue_search:
        break
    i += 1  

dt_0 and dt_1:
(-0.0010677836931061095, 0.00038691337099393925)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1     -0.0003      0.000     -0.918      0.359      -0.001       0.000
-------------------------------------------
dt_1 and dt_2:
(-0.00035490223611945237, 0.0011206246921177158)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1      0.0004      0.000      1.017      0.309      -0.000       0.001
-------------------------------------------
dt_2 and dt_3:
(-0.0008313468313829363, 0.0006340184607404785)
                          Test for equality of means                          
          

In [182]:
## dt = 54